# Edge-based graph figure

Make a nice figure to explain the edge-based graph.

In [ ]:
using Serialization
using CSV
using DataFrames
using CodecZlib
using Luxor
using LightGraphs
using MetaGraphs
using LibSpatialIndex
using Geodesy

In [ ]:
include("fw_types.jl")
include("fw_weights.jl")
include("compute_heading.jl")

In [ ]:
G = deserialize("socal.fwgr")

In [ ]:
# read names
way_names =  open("socal.fwgr.names.gz") do f
    gz = GzipDecompressorStream(f)
    DataFrame(CSV.File(gz, header=false)).Column1
end

In [ ]:
compute_edge_weights!(G, true)

In [ ]:
# build a spatial index for the vertices
function build_index(graph)
    sidx = LibSpatialIndex.RTree(2)
    for v in vertices(graph)
        vgeom = get_prop(graph, v, :geom)::LLA
        nbrs = outneighbors(graph, v)
        if length(nbrs) > 0
            # all out neighbors started life as the same node, so all will be in same
            # place geographically, just pick one
            next_geom = get_prop(graph, nbrs[1], :geom)::LLA
        else
            next_geom = vgeom
        end
        
        LibSpatialIndex.insert!(sidx, v, [min(vgeom.lat, next_geom.lat), min(vgeom.lon, next_geom.lon)], [max(vgeom.lat, next_geom.lat), max(vgeom.lon, next_geom.lon)])
    end
    return sidx
end

sidx = build_index(G)

In [ ]:
osm_bbox = [

# 34.2432
# -118.4549
# -118.4403
# 34.2342

    


34.2516
-118.5929
-118.5763
34.2413




]





pdf_output = true

In [ ]:
# up in the San Fernando Valley
bbox = [[osm_bbox[4], osm_bbox[2]], [osm_bbox[1], osm_bbox[3]]]
bbox_expand = [bbox[1] .- 0.01, bbox[2] .+ 0.01]
vs_to_draw = LibSpatialIndex.intersects(sidx, bbox_expand...)

In [ ]:
function offset_point(x, y, bearing, distance)
    pt = Point(x, y)
    off = polar(distance, (bearing - 90) * 2π / 360)
    return pt + off
end

function offset_point(xy::Point, bearing, distance)
    return offset_point(xy.x, xy.y, bearing, distance)
end

function compute_heading(origin::Point, dest::Point)
    Δ = dest - origin
    heading = atand(Δ.y, Δ.x) + 90
    # make sure it's positive
    while heading < 0
        heading += 360
    end
    return heading % 360
end

function plot_normal(graph, vs_to_draw, bbox)
    # first, figure out how to scale lon to make it look square
    lon_scale = (get_prop(graph, vs_to_draw[1], :geom)::LLA).lat |> cosd
        
    height = 500
    width = convert(Int, round((bbox[2][2] - bbox[1][2]) * lon_scale / (bbox[2][1] - bbox[1][1]) * height))
    
    @info "$height x $width"
    
    tr_lat(l) = (1 - (l - bbox[1][1]) / (bbox[2][1] - bbox[1][1])) * height
    
    # width set above to force equal aspect
    tr_lon(l) = (l - bbox[1][2]) / (bbox[2][2] - bbox[1][2]) * width
    
    if pdf_output
        d = Drawing(width, height, "../../dissertation/fig/abm/normal_graph.pdf")
    else
        d = Drawing(width, height, :svg)
    end
        
    background("white")
    
    fontface("Arial")
    fontsize(14)
    setline(1)
    sethue("black")

    for v in vs_to_draw
        origin = get_prop(graph, v, :geom)::LLA
        nbrs = outneighbors(graph, v)
        
        if length(nbrs) == 0
            continue
        end
        
        # only need to draw once, all outneighbors are in same place since this is
        # an edge-based graph and the vertex represents a street segment. but in order for
        # traversal time to be sensible, get the one closest to straight
        nbr = -1
        min_abs_angle = Inf
        
        for i in 1:length(nbrs)
            angle = abs(get_prop(graph, v, nbrs[i], :turn_angle)::Float32)
            if angle < min_abs_angle
                nbr = i
                min_abs_angle = angle
            end
        end
        
        destination = get_prop(graph, nbrs[nbr], :geom)::LLA
        
        traversal_time_rounded = convert(Int32, round(get_prop(graph, v, nbrs[nbr], :weight)::Float64))
        
        origxy = Point(tr_lon(origin.lon), tr_lat(origin.lat))
        dest = Point(tr_lon(destination.lon), tr_lat(destination.lat))
        
        bearing = compute_heading(origxy, dest)
        
        od_incr_1 = offset_point(origxy, bearing + 30, 15)
        od_incr_2 = offset_point(dest, bearing + 150, 15)
        
        
        line(origxy, od_incr_1, :stroke)
        line(od_incr_1, od_incr_2, :stroke)
        arrow(od_incr_2, dest)
                
        # label the edges
        # but only ones completely in frame
        if (origxy.x > 0 && origxy.y > 0 && origxy.x < width && origxy.y < height &&
            dest.x > 0 && dest.y > 0 && dest.x < width && dest.y < height)
            name = way_names[v]
            angle = compute_heading(origin, destination) - 90

            label = "$name $(traversal_time_rounded)s"

            if abs(angle) > 90
                angle += 180
                valign = :bottom
                halign = :right
            else
                valign = :top
                halign = :left
            end

            angle *= 2π / 360

            text(
                label,
                od_incr_1,
                angle=angle,
                valign=valign,
                halign=halign
            )
        end
    end
    
    # come back and draw nodes
    sethue("#ff7f32")
    for v in vs_to_draw
        origin = get_prop(graph, v, :geom)::LLA
        circle(Point(tr_lon(origin.lon), tr_lat(origin.lat)), 4, :fill)
    end
    
    sethue("black")
    setline(2) # half the line ends up outside plot area
    box(Point(width / 2, height / 2), width, height, :stroke)
    
        
    finish()
    preview()
end

plot_normal(G, vs_to_draw, bbox)

In [ ]:


# draw a tangent curve to the destination
function turn_to_dest(origin, bearing, dest, turn_radius)
    bearing_to_dest = compute_heading(origin, dest)
    
    turn_angle = ((bearing_to_dest - bearing + 180) % 360) - 180
    
    if turn_angle < 0
        # left hand turn to destination
        arc_center = offset_point(origin, bearing - 90, turn_radius)
        bearing_center_to_dest = compute_heading(arc_center, dest)
        dist_to_dest = distance(arc_center, dest)
        if (dist_to_dest < turn_radius)
            turn_to_dest(origin, bearing, dest, turn_radius / 2)
        else
            bearing_tangent_to_dest = bearing_center_to_dest + acosd(turn_radius / dist_to_dest)
            # offset to the right of the bearing center to dest
            tangent_end = offset_point(arc_center, bearing_tangent_to_dest, turn_radius)
            carc2r(arc_center, origin, tangent_end, :stroke)
            arrow(tangent_end, dest)
        end
    else
        # right hand turn to destination
        arc_center = offset_point(origin, bearing + 90, turn_radius)
        bearing_center_to_dest = compute_heading(arc_center, dest)
        dist_to_dest = distance(arc_center, dest)
        if (dist_to_dest < turn_radius)
            turn_to_dest(origin, bearing, dest, turn_radius / 2)
        else
            bearing_tangent_to_dest = bearing_center_to_dest - acosd(turn_radius / dist_to_dest)
            # offset to the right of the bearing center to dest
            tangent_end = offset_point(arc_center, bearing_tangent_to_dest, turn_radius)
            arc2r(arc_center, origin, tangent_end, :stroke)
            arrow(tangent_end, dest)
        end
    end
end

function plot_edgebased(graph, vs_to_draw, bbox)
    # first, figure out how to scale lon to make it look square
    lon_scale = (get_prop(graph, vs_to_draw[1], :geom)::LLA).lat |> cosd
        
    height = 500
    width = convert(Int, round((bbox[2][2] - bbox[1][2]) * lon_scale / (bbox[2][1] - bbox[1][1]) * height))
    
    @info "$height x $width"
    
    tr_lat(l) = (1 - (l - bbox[1][1]) / (bbox[2][1] - bbox[1][1])) * height
    
    # width set above to force equal aspect
    tr_lon(l) = (l - bbox[1][2]) / (bbox[2][2] - bbox[1][2]) * width
    
    function px_for_way(v)
        origin = get_prop(graph, v, :geom)::LLA

        nbrs = outneighbors(graph, v)

        # the exact geometric destination is the same for all neighbors
        if length(nbrs) > 0
            destination = get_prop(graph, nbrs[1], :geom)::LLA
            angle = compute_heading(origin, destination)
        else
            angle = 0
        end
        # first, offset the origin to the right of its actual location, so everything doesn't end up piled on top of everything
        # else
        return (offset_point(tr_lon(origin.lon), tr_lat(origin.lat), angle + 45, 25 * √2), angle)
    end
    
    if pdf_output
        d = Drawing(width, height, "../../dissertation/fig/abm/turn_based_graph.pdf")
    else
        d = Drawing(width, height, :svg)
    end
    
    background("white")
    
    fontface("Arial")
    fontsize(14)
    setline(1)
    sethue("black")

    for v in vs_to_draw
        origin_px, bearing = px_for_way(v)
        origin_spl = offset_point(origin_px, bearing, 35)
        line(origin_px, origin_spl)
        
        nbrs = outneighbors(graph, v)
        
        for nbr in nbrs
            dest_px, dbear = px_for_way(nbr)
            turn_ang = get_prop(graph, v, nbr, :turn_angle)::Float32

            if abs(abs(turn_ang % 360) - 180) < 1e-2
                continue
            end
            
            # turns go to split slightly further down
            if abs(turn_ang) > 30
                dest_px = offset_point(dest_px, dbear, 35)
            else
                dest_px = offset_point(dest_px, dbear, 25)
            end
            
            if abs(turn_ang) < 80
                offset_ang = 0
            elseif turn_ang >= 80
                # right turn, offset turn edge to right
                offset_ang = 30
            elseif turn_ang <= -80
                offset_ang = -30
            else
                error("angle not real")
            end
            
            initial_offset = offset_point(origin_spl, bearing + offset_ang, 20 * √2)
            
            # line from origin to initial_offset
            line(origin_spl, initial_offset, :stroke)
                
            # draw parallel for a while
            dist = distance(initial_offset, dest_px)
            if dist < 60
                line(initial_offset, dest_px)
                # don't lable these tiny segments
            else
                # draw a straight segment then a curve
                if turn_ang < -65
                    # left turn
                    offset_before_curve = dist - 60
                    radius = 30
                elseif turn_ang > 65
                    # right turn
                    offset_before_curve = dist - 25
                    radius = 15
                else
                    offset_before_curve = dist - 10
                    radius = 10
                end
                final_offset = offset_point(initial_offset.x, initial_offset.y, bearing, offset_before_curve)
                
                name = way_names[v]
                traversal_time_rounded = convert(Int32, round(get_prop(graph, v, nbr, :weight)::Float64))
                
                if (
                        origin_px.x > 0 && origin_px.y > 0 && origin_px.x < width && origin_px.y < height ||
                        dest_px.x > 0 && dest_px.y > 0 && dest_px.x < width && dest_px.y < height
                )
                    line(initial_offset, final_offset)
                    turn_to_dest(final_offset, bearing, dest_px, radius)
                end
                
                # only label segments entirely in bbox
                if (
                        origin_px.x > 0 && origin_px.y > 0 && origin_px.x < width && origin_px.y < height &&
                        dest_px.x > 0 && dest_px.y > 0 && dest_px.x < width && dest_px.y < height
                    )
                    if abs(turn_ang) < 30
                        label = "$name $(traversal_time_rounded)s"
                    elseif turn_ang >= 30
                        label = "$name right turn $(traversal_time_rounded)s"
                    else
                        label = "$name left turn $(traversal_time_rounded)s"
                    end 

                    angle = bearing - 90

                    if abs(angle) > 90
                        angle += 180
                        halign=:right
                    else
                        halign=:left
                    end

                    text(
                        label,
                        initial_offset,
                        angle=deg2rad(angle),
                        valign=:bottom,
                        halign=halign
                    )
                end
            end
        end
    end
    
    sethue("#ff7f32")
    for v in vs_to_draw
        origin_px, bearing = px_for_way(v)
        origin_spl = offset_point(origin_px, bearing, 35)
        circle(origin_spl, 3, :fill)
    end
        
    sethue("black")
    setline(2) # half the line ends up outside plot area
    box(Point(width / 2, height / 2), width, height, :stroke)
    
    finish()
    preview()
end

plot_edgebased(G, vs_to_draw, bbox)

In [ ]:
compute_heading(Point(0, 0), Point(0, -1))

In [ ]:
-180 % 360
